# Using Ray for Highly Parallelizable Tasks

While Ray can be used for very complex parallelization tasks,
often we just want to do something simple in parallel.
For example, we may have 100,000 time series to process with exactly the same algorithm,
and each one takes a minute of processing.

Clearly running it on a single processor is prohibitive: this would take 70 days.
Even if we managed to use 8 processors on a single machine,
that would bring it down to 9 days. But if we can use 8 machines, each with 16 cores,
it can be done in about 12 hours.

How can we use Ray for these types of task? 

We take the simple example of computing the digits of pi.
The algorithm is simple: generate random x and y, and if ``x^2 + y^2 < 1``, it's
inside the circle, we count as in. This actually turns out to be pi/4
(remembering your high school math).

The following code (and this notebook) assumes you have already set up your Ray cluster and that you are running on the head node. For more details on how to set up a Ray cluster please see [Ray Clusters Getting Started](https://docs.ray.io/en/master/cluster/getting-started.html). 


In [1]:
import ray
import random
import time
import math
from fractions import Fraction

In [7]:
# Let's start Ray
ray.init(address='auto')

2024-10-20 19:38:40,182	INFO worker.py:1601 -- Connecting to existing Ray cluster at address: 10.0.1.224:58659...


RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

We use the ``@ray.remote`` decorator to create a Ray task.
A task is like a function, except the result is returned asynchronously.

It also may not run on the local machine, it may run elsewhere in the cluster.
This way you can run multiple tasks in parallel,
beyond the limit of the number of processors you can have in a single machine.

In [8]:
@ray.remote
def pi4_sample(sample_count):
    """pi4_sample runs sample_count experiments, and returns the 
    fraction of time it was inside the circle. 
    """
    in_count = 0
    for i in range(sample_count):
        x = random.random()
        y = random.random()
        if x*x + y*y <= 1:
            in_count += 1
    return Fraction(in_count, sample_count)


To get the result of a future, we use ray.get() which 
blocks until the result is complete. 

In [9]:
SAMPLE_COUNT = 1000 * 1000
start = time.time() 
future = pi4_sample.remote(sample_count = SAMPLE_COUNT)
pi4 = ray.get(future)
end = time.time()
dur = end - start
print(f'Running {SAMPLE_COUNT} tests took {dur} seconds')

Running 1000000 tests took 0.31095314025878906 seconds


Now let's see how good our approximation is.

In [10]:
pi = pi4 * 4

In [11]:
float(pi)

3.142852

In [12]:
abs(pi-math.pi)/pi

0.0004007017862141975

Not bad at all -- we're off by roughly 4 decimal places.

Once we are done let's shut down the cluster!

In [13]:
ray.shutdown()